<a href="https://colab.research.google.com/github/YuriiSulima/ML-models-for-CO2-Reduction/blob/main/ML_Models_(Photocatalysis_on_Transition_Metals)_Version_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing all the essential libraries
import pandas as pd
import numpy as np
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
import tensorflow as tf
import xgboost as xgb
import shap
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedShuffleSplit

KeyboardInterrupt: 

In [ ]:
#Making a dataset from the table
# Read the CSV file "YS01 - ML Photocatalysis Transitional Metals.csv" and store it in the variable 'data'
data = pd.read_csv("YS01 - ML Photocatalysis Transitional Metals.csv")

# Read the CSV file "YS01 - ML Transition Metal Updated.csv" and store it in the variable 'metals'
metals = pd.read_csv("YS01 - ML Transition Metal Updated.csv")

# Print the shape of the 'data' DataFrame
print(data.shape)

# Print the shape of the 'metals' DataFrame
print(metals.shape)

In [ ]:
#Showing general statistics about our dataset
data.describe()
metals.describe()

In [ ]:
# Display the first few rows of the 'data' DataFrame
data.head()

# Display the first few rows of the 'metals' DataFrame
metals.head()

In [ ]:
data.dtypes

# Preprocessing

In [ ]:
catalyst_encoder = LabelEncoder()
type_encoder = LabelEncoder()

data.drop(['Index'],axis=1,inplace=True)
data.drop(['pH before'],axis=1,inplace=True)

data['Catalyst'] = catalyst_encoder.fit_transform(data['Catalyst'])
groups = data['Catalyst']
label_mapping = dict(zip(catalyst_encoder.classes_, catalyst_encoder.transform(catalyst_encoder.classes_)))
print(label_mapping)

data.drop(['Catalyst'],axis=1,inplace=True)
data['mmol CO / g QD'] = data['mmol CO / g QD'] / data['Time ']
data.drop(['Time '],axis=1,inplace=True)

data['Type of complex (low-spin vs high-spin)'] = catalyst_encoder.fit_transform(data['Type of complex (low-spin vs high-spin)'])

data['Magnetic Type'] = type_encoder.fit_transform(data['Magnetic Type'])
label_mapping = dict(zip(type_encoder.classes_, type_encoder.transform(type_encoder.classes_)))
print(label_mapping)
data = data.fillna(0)

features_scaler = StandardScaler()
target_scaler = StandardScaler()

data_pipeline = Pipeline([
        ('std_scaler', features_scaler),
    ])

target_pipeline = Pipeline([
        ('std_scaler', target_scaler),
    ])

target = data['mmol CO / g QD']
features = data.drop('mmol CO / g QD', axis=1)
columns_f = features.columns

features = data_pipeline.fit_transform(features)
features = pd.DataFrame(features,columns=columns_f)

target = data_pipeline.fit_transform(target.to_numpy().reshape(-1,1))
target = pd.DataFrame(target,columns=['mmol CO / g QD'])

# Preprocessing Metals Dataset

In [ ]:
names = metals['Element'].copy()

metals['Element'] = type_encoder.fit_transform(metals['Element'])
label_mapping = dict(zip(type_encoder.classes_, type_encoder.transform(type_encoder.classes_)))
print(label_mapping)

metals['Magnetic Type'] = type_encoder.fit_transform(metals['Magnetic Type'])
label_mapping = dict(zip(type_encoder.classes_, type_encoder.transform(type_encoder.classes_)))
print(label_mapping)

metals = metals.fillna(0)

columns_metals = metals.columns
metals = data_pipeline.fit_transform(metals)
metals = pd.DataFrame(metals,columns=columns_metals)


In [ ]:
metals.head()
print(columns_metals)

# Dimensionality Reduction

In [ ]:
#PCA worse
# pca = PCA(n_components=10)
# x_train = pca.fit_transform(x_train)
# X_test = pca.transform(X_test)

#T-SNE much worse
# tsne = TSNE(n_components=3, random_state=42)
# features = tsne.fit_transform(features)

# #UMAP
# pendigits = sklearn.datasets.load_digits()
# reducer = umap.UMAP(n_components=2)
# features = reducer.fit_transform(features, target)
# plt.scatter(features[:, 0], features[:, 1],s=.7)

# Stratified Split

In [ ]:
# Split the dataset into training and testing sets using GroupShuffleSplit
# split = GroupShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
# for train_index, test_index in split.split(features, target, groups):
#     x_train, X_test = features.iloc[train_index], features.iloc[test_index]
#     y_train, Y_test = target.iloc[train_index], target.iloc[test_index]

x_train, X_test, y_train, Y_test = train_test_split(features, target, test_size=0.3, random_state=0) #best states 5, 14, 21, 0
index_before = Y_test.index
Y_test = Y_test.sort_values(by='mmol CO / g QD')
sorted_indices = Y_test.index

In [ ]:
data.corr()['mmol CO / g QD'].sort_values(ascending=False)

# Functions

In [ ]:
def rmse(y_actual, y_predicted):
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    return sqrt(mean_squared_error(y_actual, y_predicted))
def plotShap(shap_values):
    # Calculate the mean absolute SHAP values
    mean_abs_shap_values = np.abs(shap_values).mean(axis=0)

    # Sort feature names and SHAP values together
    sorted_indices = np.argsort(mean_abs_shap_values)
    sorted_feature_names = columns_f[sorted_indices]
    sorted_mean_abs_shap_values = mean_abs_shap_values[sorted_indices]

    mse = mean_squared_error(Y_test, pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(Y_test, pred)
    # Plot a vertical bar chart of feature importances with values
    plt.figure(figsize=(10, 6))
    plt.figtext(0.87, 0.12, f"MSE: {mse:.5f}, RMSE: {rmse:.5f}, R2: {r2:.5f}", horizontalalignment='right')
    plt.barh(range(len(sorted_feature_names)), sorted_mean_abs_shap_values, color='blue')
    plt.yticks(range(len(sorted_feature_names)), sorted_feature_names)
    plt.xlabel('Mean Absolute SHAP Value')
    plt.title('Feature Importances with SHAP Values')
    # Add values next to the bars
    for index, value in enumerate(sorted_mean_abs_shap_values):
        plt.text(value, index, f'{value:.2f}', va='center')
    plt.show()
    return

In [ ]:
def best_hiper_search(features_train, target_train):
    parametrs_DTR = {'max_depth': range (1,13, 1)}
    parametrs_RFR = {'n_estimators': range (1, 10, 2),'max_depth': range (1,10, 1)}
    parametrs_LR = {'fit_intercept':[True,False]}

    rmse_scorer = make_scorer(rmse, greater_is_better=True)


    grid_DTR = GridSearchCV(DecisionTreeRegressor(), parametrs_DTR, cv=5, scoring=rmse_scorer)
    grid_RFR = GridSearchCV(RandomForestRegressor(), parametrs_RFR, cv=10, scoring=rmse_scorer)
    grid_LR = GridSearchCV(LinearRegression(), parametrs_LR, cv=5, scoring=rmse_scorer)


    grid_DTR.fit(features_train, target_train)
    grid_RFR.fit(features_train, target_train)
    grid_LR.fit(features_train, target_train)


    print("DecisionTreeRegressor", grid_DTR.scorer_, grid_DTR.best_score_, grid_DTR.best_params_)
    print("RandomForestRegressor", grid_RFR.scorer_, grid_RFR.best_score_, grid_RFR.best_params_)
    print("LinearRegression", grid_LR.scorer_, grid_LR.best_score_, grid_LR.best_params_)
    return

# Function to split, train, and evaluate the model
def evaluate_random_state_xgb(random_state):
    # Split the dataset using train_test_split
    X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size=0.3, random_state=random_state)

    # Train an XGBoost model
    model = xgb.XGBRegressor(objective='reg:squarederror', random_state=random_state)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    return mse, rmse, r2
# Function to split, train, and evaluate the model
def evaluate_random_state_dtr(random_state):
    # Split the dataset using train_test_split
    X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size=0.3, random_state=random_state)

    # Train an DTR model
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    return mse, rmse, r2
def evaluate_random_state_rfr(random_state):
    # Split the dataset using train_test_split
    X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size=0.3, random_state=random_state)

    # Train an RFR model
    model = RandomForestRegressor()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    return mse, rmse, r2
def display_shap_importance_values():
    # Calculate the mean absolute SHAP values for each feature
    shap_importance = np.abs(shap_values).mean(axis=0)
    columns = columns_f.to_list()
    print(columns)

    shap_importance_df = pd.DataFrame({
        'Feature': columns,
        'SHAP Importance': shap_importance
    }).sort_values(by='SHAP Importance', ascending=False)
    #print("Feature importance based on mean absolute SHAP values:\n", shap_importance_df)
    shap_importance_df = shap_importance_df.set_index('Feature')
    shap_importance_df = shap_importance_df.T
    # Compute the score using a dictionary lookup for SHAP values with a default weight of 0
    metals['Score'] = metals.apply(
        lambda row: sum(row[feature] * shap_importance_df.get(feature, 0) for feature in columns_metals if feature != 'Element'), axis=1
    )
    combined_df = pd.DataFrame({
    'Name': names,
    'Score': metals['Score']
    })
    combined_df = combined_df.sort_values(by='Score')
    print(combined_df)

print("Selection of hyperparameters:")
#best_hiper_search(x_train, y_train)
print('')


# Linear Regression

In [ ]:
model = LinearRegression(fit_intercept = True)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, x_train, y_train,
                                scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
display_scores(rmse_scores)


In [ ]:
model.fit(x_train, y_train)
pred = model.predict(X_test)

#Sorting
dictionary = {index: prediction for index, prediction in zip(index_before, pred)}
pred = np.array([dictionary[idx] for idx in sorted_indices])

print('RMSE: ', mean_squared_error(Y_test, pred)**0.5)
#Calculation of Mean Absolute Error
mae = mean_absolute_error(Y_test, pred)
print('Mean Absolute Error of the model is: ', mae)

In [ ]:
accuracy = model.score(X_test, Y_test)
print("Аccuracy", ":", accuracy)
r2 = r2_score(Y_test, pred)
print("R2: ", r2)

In [ ]:
plt.figure(figsize=(8, 6))

# Plot true values
plt.plot(range(len(Y_test)), Y_test, label='True Values', marker='o')

# Plot predicted values
plt.plot(range(len(pred)), pred, label='Predicted Values', marker='o')


# Add labels and legend
plt.xlabel('Data Points')
plt.ylabel('Values')
plt.title('True vs. Predicted Values')
plt.legend()

plt.show()

# Decision Tree Regressor

> Add blockquote



In [ ]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)
pred = model.predict(X_test)

#Sorting
dictionary = {index: prediction for index, prediction in zip(index_before, pred)}
pred = np.array([dictionary[idx] for idx in sorted_indices])

print('RMSE: ', mean_squared_error(Y_test, pred)**0.5)
#Calculation of Mean Absolute Error
mae = mean_absolute_error(Y_test, pred)
print('Mean Absolute Error of the model is: ', mae)
accuracy = model.score(X_test, Y_test)

scores = cross_val_score(model, x_train, y_train,
                                scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
display_scores(rmse_scores)
print("Аccuracy", ":", accuracy)
r2 = r2_score(Y_test, pred)
print("R2: ", r2)
plt.figure(figsize=(8, 6))

# Plot true values
plt.plot(range(len(Y_test)), Y_test, label='True Values', marker='o')

# Plot predicted values
plt.plot(range(len(pred)), pred, label='Predicted Values', marker='o')


# Add labels and legend
plt.xlabel('Data Points')
plt.ylabel('Values')
plt.title('True vs. Predicted Values')
plt.legend()

plt.show()
#metals["DTR"] =

In [ ]:
# Create a SHAP explainer object
explainer = shap.TreeExplainer(model)

# Compute SHAP values for the test set
shap_values = explainer.shap_values(X_test)

# Compute the base value (expected value of the model output)
base_value = explainer.expected_value
#shap.summary_plot(shap_values, X_test, feature_names=columns_f)
plotShap(shap_values)
display_shap_importance_values()

In [ ]:
# Try different random states and record performance
results = []
for random_state in range(50):  # Try 50 different random states
    mse, rmse, r2 = evaluate_random_state_dtr(random_state)
    results.append((random_state, mse, rmse, r2))

# Convert results to a DataFrame
results_df = pd.DataFrame(results, columns=['Random State', 'MSE', 'RMSE', 'R²'])

# Find the best random state based on RMSE
best_random_state = results_df.loc[results_df['RMSE'].idxmin()]

print("Best Random State:")
print(best_random_state)

# Optionally, plot the results
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(results_df['Random State'], results_df['RMSE'], marker='o')
plt.title('RMSE for Different Random States')
plt.xlabel('Random State')
plt.ylabel('RMSE')
plt.grid(True)
plt.show()


# Random Forest Regressor

In [ ]:
model = RandomForestRegressor(max_depth = 6)
model.fit(x_train, y_train)
pred = model.predict(X_test)

#Sorting
dictionary = {index: prediction for index, prediction in zip(index_before, pred)}
pred = np.array([dictionary[idx] for idx in sorted_indices])

print('RMSE: ', mean_squared_error(Y_test, pred)**0.5)
#Calculation of Mean Absolute Error
mae = mean_absolute_error(Y_test, pred)
print('Mean Absolute Error of the model is: ', mae)
accuracy = model.score(X_test, Y_test)
print("Аccuracy", ":", accuracy)
r2 = r2_score(Y_test, pred)
print("R2: ", r2)
scores = cross_val_score(model, x_train, y_train,
                                scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
display_scores(rmse_scores)
plt.figure(figsize=(8, 6))

# Plot true values
plt.plot(range(len(Y_test)), Y_test, label='True Values', marker='o',markersize=12)

# Plot predicted values
plt.plot(range(len(pred)), pred, label='Predicted Values', marker='o',markersize=12)


# Add labels and legend
plt.xlabel('Data Points', fontsize = 25)
plt.ylabel('Values', fontsize = 25)
plt.xticks(fontsize = 22)
plt.yticks(fontsize = 22)
plt.title('True vs. Predicted Values', fontsize = 25)
plt.legend(fontsize = 14)

plt.show()

In [ ]:
# Create a SHAP explainer object
explainer = shap.TreeExplainer(model)

# Compute SHAP values for the test set
shap_values = explainer.shap_values(X_test)

# Compute the base value (expected value of the model output)
base_value = explainer.expected_value
#shap.summary_plot(shap_values, X_test, feature_names=columns_f)
#shap.summary_plot(shap_values, X_test, feature_names=columns_f, plot_type="bar")
plotShap(shap_values)
display_shap_importance_values()

In [ ]:
# Try different random states and record performance
results = []
for random_state in range(50):  # Try 50 different random states
    mse, rmse, r2 = evaluate_random_state_rfr(random_state)
    results.append((random_state, mse, rmse, r2))

# Convert results to a DataFrame
results_df = pd.DataFrame(results, columns=['Random State', 'MSE', 'RMSE', 'R²'])

# Find the best random state based on RMSE
best_random_state = results_df.loc[results_df['RMSE'].idxmin()]

print("Best Random State:")
print(best_random_state)

# Optionally, plot the results
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(results_df['Random State'], results_df['RMSE'], marker='o')
plt.title('RMSE for Different Random States')
plt.xlabel('Random State')
plt.ylabel('RMSE')
plt.grid(True)
plt.show()


# SVR

In [ ]:
# model = SVR(kernel='rbf', C=100, gamma=0.1)
# model.fit(x_train, y_train)
# pred = model.predict(X_test)

# #Sorting
# dictionary = {index: prediction for index, prediction in zip(index_before, pred)}
# pred = np.array([dictionary[idx] for idx in sorted_indices])

# print('RMSE: ', mean_squared_error(Y_test, pred)**0.5)
# #Calculation of Mean Absolute Error
# mae = mean_absolute_error(Y_test, pred)
# print('Mean Absolute Error of the model is: ', mae)
# accuracy = model.score(X_test, Y_test)
# print("Аccuracy", ":", accuracy)
# r2 = r2_score(Y_test, pred)
# print("R2: ", r2)
# scores = cross_val_score(model, x_train, y_train,
#                                 scoring="neg_mean_squared_error", cv=10)
# rmse_scores = np.sqrt(-scores)
# display_scores(rmse_scores)
# plt.figure(figsize=(8, 6))

# # Plot true values
# plt.plot(range(len(Y_test)), Y_test, label='True Values', marker='o')

# # Plot predicted values
# plt.plot(range(len(pred)), pred, label='Predicted Values', marker='o')


# # Add labels and legend
# plt.xlabel('Data Points')
# plt.ylabel('Values')
# plt.title('True vs. Predicted Values')
# plt.legend()

# plt.show()

# Lasso Model

In [ ]:
# lasso_reg = Lasso(alpha=0.1)
# lasso_reg.fit(x_train, y_train)
# pred = model.predict(X_test)
# print('RMSE: ', mean_squared_error(Y_test, pred)**0.5)
# #Calculation of Mean Absolute Error
# mae = mean_absolute_error(Y_test, pred)
# print('Mean Absolute Error of the model is: ', mae)
# accuracy = model.score(X_test, Y_test)
# print("Аccuracy", ":", accuracy)

# scores = cross_val_score(model, x_train, y_train,
#                                 scoring="neg_mean_squared_error", cv=10)
# rmse_scores = np.sqrt(-scores)
# display_scores(rmse_scores)
# plt.figure(figsize=(8, 6))

# # Plot true values
# plt.plot(range(len(Y_test)), Y_test, label='True Values', marker='o')

# # Plot predicted values
# plt.plot(range(len(pred)), pred, label='Predicted Values', marker='o')


# # Add labels and legend
# plt.xlabel('Data Points')
# plt.ylabel('Values')
# plt.title('True vs. Predicted Values')
# plt.legend()

# plt.show()

# Elastic Net

In [ ]:
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
elastic_net.fit(x_train, y_train)
pred = model.predict(X_test)

#Sorting
dictionary = {index: prediction for index, prediction in zip(index_before, pred)}
pred = np.array([dictionary[idx] for idx in sorted_indices])

print('RMSE: ', mean_squared_error(Y_test, pred)**0.5)
#Calculation of Mean Absolute Error
mae = mean_absolute_error(Y_test, pred)
print('Mean Absolute Error of the model is: ', mae)
accuracy = model.score(X_test, Y_test)
print("Аccuracy", ":", accuracy)
r2 = r2_score(Y_test, pred)
print("R2: ", r2)
scores = cross_val_score(model, x_train, y_train,
                                scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
display_scores(rmse_scores)
plt.figure(figsize=(8, 6))

# Plot true values
plt.plot(range(len(Y_test)), Y_test, label='True Values', marker='o')

# Plot predicted values
plt.plot(range(len(pred)), pred, label='Predicted Values', marker='o')


# Add labels and legend
plt.xlabel('Data Points')
plt.ylabel('Values')
plt.title('True vs. Predicted Values')
plt.legend()

plt.show()

# XGB Regressor

In [ ]:
model = xgb.XGBRegressor(objective='reg:squarederror')
model.fit(x_train, y_train)
pred = model.predict(X_test)

#Sorting
dictionary = {index: prediction for index, prediction in zip(index_before, pred)}
pred = np.array([dictionary[idx] for idx in sorted_indices])

print('RMSE: ', mean_squared_error(Y_test, pred)**0.5)
#Calculation of Mean Absolute Error
mae = mean_absolute_error(Y_test, pred)
print('Mean Absolute Error of the model is: ', mae)
accuracy = model.score(X_test, Y_test)
print("Аccuracy", ":", accuracy)
r2 = r2_score(Y_test, pred)
print("R2: ", r2)
scores = cross_val_score(model, x_train, y_train,
                                scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
display_scores(rmse_scores)
plt.figure(figsize=(8, 6))

# Plot true values
plt.plot(range(len(Y_test)), Y_test, label='True Values', marker='o')

# Plot predicted values
plt.plot(range(len(pred)), pred, label='Predicted Values', marker='o')


# Add labels and legend
plt.xlabel('Data Points')
plt.ylabel('Values')
plt.title('True vs. Predicted Values')
plt.legend()

plt.show()

In [ ]:
# Create a SHAP explainer object
explainer = shap.TreeExplainer(model)

# Compute SHAP values for the test set
shap_values = explainer.shap_values(X_test)

# Compute the base value (expected value of the model output)
base_value = explainer.expected_value
#shap.summary_plot(shap_values, X_test, feature_names=columns_f)
plotShap(shap_values)
display_shap_importance_values()

In [ ]:
# Try different random states and record performance
results = []
for random_state in range(50):  # Try 50 different random states
    mse, rmse, r2 = evaluate_random_state_xgb(random_state)
    results.append((random_state, mse, rmse, r2))

# Convert results to a DataFrame
results_df = pd.DataFrame(results, columns=['Random State', 'MSE', 'RMSE', 'R²'])

# Find the best random state based on RMSE
best_random_state = results_df.loc[results_df['RMSE'].idxmin()]

print("Best Random State:")
print(best_random_state)

# Optionally, plot the results
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(results_df['Random State'], results_df['RMSE'], marker='o')
plt.title('RMSE for Different Random States')
plt.xlabel('Random State')
plt.ylabel('RMSE')
plt.grid(True)
plt.show()


# Ensemble

In [ ]:
dtr = DecisionTreeRegressor(random_state=42)
rfr = RandomForestRegressor(random_state=42)
voter = VotingRegressor(
    estimators=[('dtr', dtr), ('rfr', rfr)])
voter.fit(x_train, y_train)
pred = voter.predict(X_test)

#Sorting
dictionary = {index: prediction for index, prediction in zip(index_before, pred)}
pred = np.array([dictionary[idx] for idx in sorted_indices])

print('RMSE: ', mean_squared_error(Y_test, pred)**0.5)
#Calculation of Mean Absolute Error
mae = mean_absolute_error(Y_test, pred)
print('Mean Absolute Error of the model is: ', mae)
accuracy = model.score(X_test, Y_test)
print("Аccuracy", ":", accuracy)
r2 = r2_score(Y_test, pred)
print("R2: ", r2)
scores = cross_val_score(model, x_train, y_train,
                                scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
display_scores(rmse_scores)
plt.figure(figsize=(8, 6))

# Plot true values
plt.plot(range(len(Y_test)), Y_test, label='True Values', marker='o')

# Plot predicted values
plt.plot(range(len(pred)), pred, label='Predicted Values', marker='o')


# Add labels and legend
plt.xlabel('Data Points')
plt.ylabel('Values')
plt.title('True vs. Predicted Values')
plt.legend()

plt.show()



```
# This is formatted as code
```

# Bagging Regressor

In [ ]:
bag = BaggingRegressor(
    VotingRegressor(estimators=[('dtr', dtr), ('rfr', rfr)]), n_estimators=500,
    max_samples=30, bootstrap=True, n_jobs=-1, random_state=42)
bag.fit(x_train, y_train)
pred = bag.predict(X_test)

#Sorting
dictionary = {index: prediction for index, prediction in zip(index_before, pred)}
pred = np.array([dictionary[idx] for idx in sorted_indices])

print('RMSE: ', mean_squared_error(Y_test, pred)**0.5)
#Calculation of Mean Absolute Error
mae = mean_absolute_error(Y_test, pred)
print('Mean Absolute Error of the model is: ', mae)
accuracy = model.score(X_test, Y_test)
print("Аccuracy", ":", accuracy)
r2 = r2_score(Y_test, pred)
print("R2: ", r2)
scores = cross_val_score(model, x_train, y_train,
                                scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
display_scores(rmse_scores)
plt.figure(figsize=(8, 6))

# Plot true values
plt.plot(range(len(Y_test)), Y_test, label='True Values', marker='o')

# Plot predicted values
plt.plot(range(len(pred)), pred, label='Predicted Values', marker='o')


# Add labels and legend
plt.xlabel('Data Points')
plt.ylabel('Values')
plt.title('True vs. Predicted Values')
plt.legend()

plt.show()

# # Calculate differences
# difference = [p - t for p, t in zip(pred, Y_test)]

# # Create DataFrame
# dt = {'True Values': Y_test,
#         'Predicted Values': pred,
#         'Difference': difference}

# df = pd.DataFrame(dt)
# df

# Perceptron (ANN)

In [ ]:
#per = Perceptron(max_iter=100, random_state=42)
#per.fit(x_train, y_train)

# DNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, Dropout

model = Sequential([
    Dense(100,input_shape=(21,)),
    LeakyReLU(alpha=0.1),
    Dropout(0.1),
    Dense(100),
    LeakyReLU(alpha=0.1),
    Dropout(0.1),
    Dense(100),
    LeakyReLU(alpha=0.1),
    Dropout(0.1),
    Dense(3)
])

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(x_train, y_train, epochs=500, batch_size=32, validation_split=0.2)

loss = model.evaluate(X_test, Y_test)
print("Test Loss:", loss)

pred = model.predict(X_test)

rmse_scores = np.sqrt(-scores)
display_scores(rmse_scores)
plt.figure(figsize=(8, 6))

# Plot true values
plt.plot(range(len(Y_test)), Y_test, label='True Values', marker='o')

# Plot predicted values
plt.plot(range(len(pred)), pred, label='Predicted Values', marker='o')


# Add labels and legend
plt.xlabel('Data Points')
plt.ylabel('Values')
plt.title('True vs. Predicted Values')
plt.legend()

plt.show()



# RNN (Recurrent Neural Network)

In [ ]:
# from tensorflow.keras.layers import SimpleRNN
# model = Sequential([
#     SimpleRNN(64, input_shape=(10,1)),  # RNN layer with 64 neurons
#     Dropout(0.2),  # Dropout layer to prevent overfitting
#     SimpleRNN(64),  # RNN layer with 64 neurons
#     Dropout(0.2),  # Dropout layer to prevent overfitting
#     SimpleRNN(32),  # RNN layer with 32 neurons
#     Dropout(0.2),  # Dropout layer to prevent overfitting
#     Dense(3)  # Output layer with no activation function for regression
# ])

# model.compile(optimizer='adam', loss='mean_squared_error')

# X_train_RNN = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
# X_test_RNN = x_test.reshape((X_test.shape[0], X_test.shape[1], 1))
# Y_train_RNN = y_train.reshape((y_train.shape[0], y_train.shape[1], 1))
# Y_test_RNN = y_test.reshape((Y_test.shape[0], Y_test.shape[1], 1))

# model.fit(X_train_RNN, Y_train_RNN, epochs=50, batch_size=32, validation_split=0.2)

# loss = model.evaluate(X_test_RNN, Y_test_RNN)
# print("Test Loss:", loss)

# pred = model.predict(X_test_RNN)

# rmse_scores = np.sqrt(-scores)
# display_scores(rmse_scores)
# plt.figure(figsize=(8, 6))

# # Plot true values
# plt.plot(range(len(Y_test)), Y_test, label='True Values', marker='o')

# # Plot predicted values
# plt.plot(range(len(pred)), pred, label='Predicted Values', marker='o')


# # Add labels and legend
# plt.xlabel('Data Points')
# plt.ylabel('Values')
# plt.title('True vs. Predicted Values')
# plt.legend()

# plt.show()


# Prediction

In [ ]:
pred = voter.predict([[1,10,1,0,0,0,0,0,0]])
pred = pred.reshape(1,-1)
data_pipeline.inverse_transform(pred)


# RL Model